In [5]:
#pip install pandas mysqlclient

In [9]:

"""
connection = MySQLdb.connect(
  host= os.getenv("DB_HOST"),
  user=os.getenv("DB_USERNAME"),
  passwd= os.getenv("DB_PASSWORD"),
  db= os.getenv("DB_NAME"),
  #autocommit = True,
  #ssl_mode = "VERIFY_IDENTITY",
  ssl = {
    #"ca": "/etc/ssl/cert.pem",
    "rejectedUnauthorized": False
  }
)
"""

'\nconnection = MySQLdb.connect(\n  host= os.getenv("DB_HOST"),\n  user=os.getenv("DB_USERNAME"),\n  passwd= os.getenv("DB_PASSWORD"),\n  db= os.getenv("DB_NAME"),\n  #autocommit = True,\n  #ssl_mode = "VERIFY_IDENTITY",\n  ssl = {\n    #"ca": "/etc/ssl/cert.pem",\n    "rejectedUnauthorized": False\n  }\n)\n'

In [10]:
from dotenv import load_dotenv
load_dotenv()
import os
import MySQLdb
import pandas as pd


In [21]:

connection = MySQLdb.connect(
  host= os.getenv("DB_HOST"),
  user=os.getenv("DB_USERNAME"),
  passwd= os.getenv("DB_PASSWORD"),
  db= os.getenv("DB_NAME"), 
  ssl = { 
    "rejectedUnauthorized": False
  }
)

In [22]:
# Crea un cursor para ejecutar consultas
cursor = connection.cursor()

# Ejecuta la consulta "USE meson;" para seleccionar la base de datos "meson"
cursor.execute("USE meson;")

# Ejecuta la consulta "SHOW TABLES;" para mostrar las tablas en la base de datos
cursor.execute("SHOW TABLES;") 

# Obtiene los resultados de la consulta "SHOW TABLES;"
tables = cursor.fetchall()

# Imprime la lista de tablas
print("Tablas en la base de datos 'meson':")
for tab in tables:
    print(tab[0])

# Cierra el cursor y la conexión
#cursor.close()
#onnection.close()

Tablas en la base de datos 'meson':
dolar_price
meli_cars


In [24]:
df = pd.read_csv('./dolar.csv')
df = df.drop("Unnamed: 0", axis=1)

In [25]:
# Nombre de la tabla en la base de datos
table_name = "dolar_price"

# Inserta el DataFrame en la tabla
#df.to_sql(table_name, con=connection, if_exists="append", index=False)


In [26]:
# Nombre de la tabla en la base de datos
table_name = "dolar_price"

# Reemplaza los valores NaN con 0
df = df.fillna(0)

# Convierte el DataFrame a una lista de tuplas
data = [tuple(row) for row in df.values]

# Define la sentencia SQL de inserción
insert_query = "INSERT INTO {} (Date, Blue_Sell, Oficial_Sell, Blue_Buy, Oficial_Buy) VALUES (%s, %s, %s, %s, %s)".format(table_name)

try:
    # Ejecuta la sentencia SQL de inserción
    with connection.cursor() as cursor:
        cursor.executemany(insert_query, data)

    # Confirma los cambios en la base de datos
    connection.commit()

    print("Datos insertados con éxito.")
except MySQLdb.Error as e:
    # En caso de error, deshace cualquier cambio pendiente y muestra un mensaje de error
    connection.rollback()
    print("Error al insertar datos:", e)

# Cierra la conexión
#connection.close()

Datos insertados con éxito.


In [27]:
# Define la consulta SQL que deseas ejecutar
sql_query = "SELECT * FROM dolar_price"

# Ejecuta la consulta y almacena el resultado en un DataFrame
dfx = pd.read_sql(sql_query, connection)
dfx

/tmp/ipykernel_5008/628907565.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfx = pd.read_sql(sql_query, connection)


,id,Date,Blue_Sell,Oficial_Sell,Blue_Buy,Oficial_Buy
0,1,2011-01-03,4.0,4.0,4.0,4.0
1,2,2011-01-04,4.0,4.0,4.0,4.0
2,3,2011-01-05,4.0,4.0,4.0,4.0
3,4,2011-01-06,4.0,4.0,4.0,4.0
4,5,2011-01-07,4.0,4.0,4.0,4.0
...,...,...,...,...,...,...
7731,7732,2023-10-02,790.0,347.0,800.0,367.0
7732,7733,2023-10-03,800.0,347.0,810.0,367.0
7733,7734,2023-10-04,833.0,347.0,843.0,367.0
7734,7735,2023-10-05,833.0,347.0,843.0,367.0
